In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/joaquin/Documents/GitHub/skforecast


In [2]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [3]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

In [4]:
range_detector = RangeDriftDetector()
range_detector._get_features_range(X=y)

(np.float64(4.647973705440567e-05), np.float64(0.9998058149870046))

In [5]:
range_detector.fit(series=series_dict)
range_detector.series_values_range_

{'series_1': (np.float64(0.0003101521347325109),
  np.float64(0.9998914836078481)),
 'series_2': (np.float64(2.1364093648723603e-05),
  np.float64(0.9998881493412435))}

In [6]:
range_detector.fit(series=series_dict, exog=exog)
range_detector.series_values_range_
pprint(range_detector.exog_values_range_)

{'temp': (np.float64(0.00018004770333068798), np.float64(0.9999101717159763))}


# FIT

In [7]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(4.647973705440567e-05), np.float64(0.9998058149870046))}
['y']


In [8]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0003101521347325109),
              np.float64(0.9998914836078481)),
 'series_2': (np.float64(2.1364093648723603e-05),
              np.float64(0.9998881493412435))}
['series_1', 'series_2']


In [9]:
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0003101521347325109), np.float64(0.9998914836078481)), 'series_2': (np.float64(2.1364093648723603e-05), np.float64(0.9998881493412435))} 
Exogenous value ranges: {} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: [] 
Series data type: <class 'pandas.core.frame.DataFrame'> 
Exogenous data type: None 

In [10]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0003101521347325109),
              np.float64(0.9998914836078481)),
 'series_2': (np.float64(2.1364093648723603e-05),
              np.float64(0.9998881493412435))}
['series_1', 'series_2']


In [11]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0003101521347325109),
              np.float64(0.9998914836078481)),
 'series_2': (np.float64(2.1364093648723603e-05),
              np.float64(0.9998881493412435))}
['series_1', 'series_2']


In [12]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.00018004770333068798), np.float64(0.9999101717159763))}
['temp']


In [13]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'A', 'C', 'B'},
 'cat_feature_1': {'A', 'C', 'B'},
 'cat_feature_2': {'C', 'B', 'A'},
 'cat_feature_3': {'A', 'C', 'B'},
 'cat_feature_4': {'C', 'B', 'A'},
 'feature_0': (np.float64(0.00022554969720312013),
               np.float64(0.9999851127553543)),
 'feature_1': (np.float64(0.0004100942413198805),
               np.float64(0.9984838095587901)),
 'feature_2': (np.float64(0.00017482645124622387),
               np.float64(0.9999487339353215)),
 'feature_3': (np.float64(0.00013896554521097837),
               np.float64(0.9993596030059192)),
 'feature_4': (np.float64(3.376854202308266e-05),
               np.float64(0.9997414078569403))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [14]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'A', 'C', 'B'},
              'cat_feature_1': {'A', 'C', 'B'},
              'cat_feature_2': {'C', 'B', 'A'},
              'cat_feature_3': {'A', 'C', 'B'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.00022554969720312013),
                            np.float64(0.9999851127553543)),
              'feature_1': (np.float64(0.0004100942413198805),
                            np.float64(0.9984838095587901)),
              'feature_2': (np.float64(0.00017482645124622387),
                            np.float64(0.9999487339353215)),
              'feature_3': (np.float64(0.00013896554521097837),
                            np.float64(0.9993596030059192)),
              'feature_4': (np.float64(3.376854202308266e-05),
                            np.float64(0.9997414078569403))},
 'series_2': {'cat_feature_0': {'A', 'C', 'B'},
              'cat_feature_1': {'A', 'C', 'B'},
              'cat_feature_2': {'C', 'B'

In [15]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'A', 'C', 'B'},
              'cat_feature_1': {'A', 'C', 'B'},
              'cat_feature_2': {'C', 'B', 'A'},
              'cat_feature_3': {'A', 'C', 'B'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.00022554969720312013),
                            np.float64(0.9999851127553543)),
              'feature_1': (np.float64(0.0004100942413198805),
                            np.float64(0.9984838095587901)),
              'feature_2': (np.float64(0.00017482645124622387),
                            np.float64(0.9999487339353215)),
              'feature_3': (np.float64(0.00013896554521097837),
                            np.float64(0.9993596030059192)),
              'feature_4': (np.float64(3.376854202308266e-05),
                            np.float64(0.9997414078569403))},
 'series_2': {'cat_feature_0': {'A', 'C', 'B'},
              'cat_feature_1': {'A', 'C', 'B'},
              'cat_feature_2': {'C', 'B'

# Predict

In [16]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'y': (np.float64(4.647973705440567e-05), np.float64(0.9998058149870046))}
{'temp': (np.float64(0.00018004770333068798), np.float64(0.9999101717159763))}


In [17]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
range_detector.predict(last_window=last_window_valid)
range_detector.predict(last_window=last_window_invalid)


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y': 'y' has one or more values outside the range seen during training [0.00005,     │
│ 0.99981]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [18]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'series_1' has one or more values outside the range seen during training │
│ [0.00031, 0.99989]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'series_2' has one or more values outside the range seen during training │
│ [0.00002, 0.99989]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)

In [20]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

In [21]:
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

KeyError: 'temp'

In [ ]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0': 'feature_0' has one or more values outside the range seen during        │
│ training [0.00008, 0.99991]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1': 'feature_1' has one or more values outside the range seen during        │
│ training [0.00002, 0.99994]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0': 'feature_0' has one or more values outside the range seen during        │
│ training [0.00008, 0.99991]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1': 'feature_1' has one or more values outside the range seen during        │
│ training [0.00002, 0.99994]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
range_detector = RangeDriftDetector()
range_detector.fit(series=series_dict, exog=exog_dict)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00008, 0.99991]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00002, 0.99994]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00008, 0.99991]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00002, 0.99994]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :194                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯